In [1]:
import pandas as pd
from histr import Shabdansh
from typing import List

In [2]:
def get_top_n_names(data_path: str = '../naam.csv', n:int=32000) -> List[str]:
    names = pd.read_csv(data_path)
    names['name'] = names['name'].map(Shabdansh)
    # remove names with only one syllable
    names = names[names['name'].map(len) > 1]
    names['is_valid_name'] = names['name'].apply(lambda name: all(map(Shabdansh.is_valid_cluster, list(name))))
    names = names[names['is_valid_name']]
    words = names.sort_values(by=['count'], ascending=False)['name'][:32000].tolist()
    return words

In [3]:
names = get_top_n_names()

/tmp/ipykernel_3235403/636747633.py:8: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  words = names.sort_values(by=['count'], ascending=False)['name'][:32000].tolist()


In [4]:
names[:50]

['राम',
 'पुत्र',
 'राजेन्द्र',
 'राजाराम',
 'ओमप्रकाश',
 'अब्दुल',
 'छोटेलाल',
 'सीताराम',
 'जगदीश',
 'राजेश',
 'राधेश्याम',
 'श्रीराम',
 'सुरेश',
 'अशोक',
 'रमेश',
 'रामप्रसाद',
 'रामनाथ',
 'राजकुमार',
 'पत्नी',
 'विजय',
 'राकेश',
 'सुरेन्द्र',
 'बाबूराम',
 'रामनरेश',
 'रामलाल',
 'शिव',
 'महेन्द्र',
 'दिनेश',
 'रामचन्द्र',
 'श्यामलाल',
 'विनोद',
 'विश्वनाथ',
 'रामलखन',
 'अनिल',
 'रामपाल',
 'रामकुमार',
 'रामऔतार',
 'रामदेव',
 'रामदास',
 'रामूप',
 'बाबूलाल',
 'संजय',
 'कल्लू',
 'हीरालाल',
 'बैजनाथ',
 'कमला',
 'सियाराम',
 'रामेश्वर',
 'रामचन्दर',
 'जगन्नाथ']

In [5]:
print(f"Min Syllable Count: {min(len(w) for w in names)}, Max Syllable Count: {max(len(w) for w in names)}")

Min Syllable Count: 2, Max Syllable Count: 10


In [6]:
b = {}
for w in names:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1

In [7]:
sorted(b.items(), key = lambda kv: -kv[1])

[(('न', '<E>'), 3430),
 (('ह', '<E>'), 3272),
 (('र', '<E>'), 2900),
 (('ल', '<E>'), 2397),
 (('रा', 'म'), 2131),
 (('सिं', 'ह'), 1944),
 (('<S>', 'रा'), 1822),
 (('<S>', 'अ'), 1661),
 (('म', '<E>'), 1313),
 (('<S>', 'स'), 1239),
 (('ला', 'ल'), 1157),
 (('सि', 'ह'), 1124),
 (('द', '<E>'), 1108),
 (('<S>', 'म'), 1018),
 (('<S>', 'ज'), 860),
 (('<S>', 'सु'), 832),
 (('<S>', 'र'), 800),
 (('<S>', 'ह'), 755),
 (('पा', 'ल'), 732),
 (('री', '<E>'), 730),
 (('ली', '<E>'), 705),
 (('<S>', 'ब'), 678),
 (('<S>', 'वि'), 647),
 (('<S>', 'क'), 643),
 (('त', '<E>'), 635),
 (('<S>', 'न'), 633),
 (('<S>', 'मु'), 620),
 (('श', '<E>'), 610),
 (('रा', 'ज'), 555),
 (('कु', 'मा'), 543),
 (('ई', '<E>'), 536),
 (('<S>', 'इ'), 521),
 (('ती', '<E>'), 508),
 (('ज', '<E>'), 497),
 (('मा', 'र'), 495),
 (('सा', 'द'), 489),
 (('प्र', 'सा'), 477),
 (('न्द्र', '<E>'), 449),
 (('दी', 'न'), 441),
 (('<S>', 'शि'), 434),
 (('<S>', 'प'), 426),
 (('रा', 'य'), 425),
 (('र', 'न'), 422),
 (('ना', 'थ'), 421),
 (('थ', '<E>'), 4

In [8]:
chars_ls = [list(name) for name in names]
chars = []
for ls in chars_ls:
    chars += ls

In [9]:
purna_virama = '।'
chars = sorted(list(set(chars)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi[purna_virama] = 0
itos = {i:s for s,i in stoi.items()}
unique_syallable_count = len(stoi)

In [10]:
# GOAL: maximize likelihood of the data w.r.t. model parameters (statistical modeling)
# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood

# log(a*b*c) = log(a) + log(b) + log(c)

In [11]:
# --------- !!! OPTIMIZATION !!! yay, but this time actually --------------

In [12]:
import jax
import jax.numpy as jnp
from jax import random

In [13]:
# create the dataset
xs, ys = [], []
for w in names:
  chs = [purna_virama] + list(w) + [purna_virama]
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = jnp.array(xs)
ys = jnp.array(ys)
num = xs.shape[0]
print('number of examples: ', num)

number of examples:  161660


In [14]:
import torch
import torch.nn.functional as F

# finally, sample from the 'neural net' model
def generate_names(W: torch.tensor, name_count: int = 5) -> None:
    g = torch.Generator().manual_seed(2147483647)
    for i in range(name_count):
        out = []
        ix = 0
        while True:
            xenc = F.one_hot(torch.tensor([ix]), num_classes=unique_syallable_count).float()
            logits = xenc @ W # predict log-counts
            counts = logits.exp() # counts, equivalent to N
            p = counts / counts.sum(1, keepdims=True) # probabilities for next character
            ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
            out.append(itos[ix])
            if ix == 0:
                break
        print(''.join(out))

In [19]:
jax.nn.one_hot(xs, num_classes=unique_syallable_count).shape

(161660, 1296)

In [15]:
from jax import grad, jit
import jax.numpy as jnp
import numpy as np


# Define the loss function
def loss_fn(W, xs, ys):
    xenc = jax.nn.one_hot(xs, num_classes=unique_syallable_count)
    logits = xenc @ W
    counts = jnp.exp(logits)
    probs = counts / jnp.sum(counts, axis=1, keepdims=True)
    loss = -jnp.mean(jnp.log(probs[jnp.arange(num), ys])) + 0.01*(jnp.power(W, 2)).mean()
    return loss

# Define the update function
@jit
def update(W, xs, ys):
    grad_fn = jit(grad(loss_fn))
    loss = loss_fn(W, xs, ys)
    dW = grad_fn(W, xs, ys)
    W -= 50 * dW
    return W, loss

In [16]:
from jax_smi import initialise_tracking
initialise_tracking()

seed = 2147483647
key = random.PRNGKey(seed)
key, subkey = random.split(key)

# generate random numbers with mean 0 and standard deviation 1
W = random.normal(subkey, (unique_syallable_count, unique_syallable_count))
for k in range(10000):
    W, loss = update(W, xs, ys)
    
    if k%100 == 0:
        print(f"At step: {k}, loss: {loss}")
        W_hat = np.asarray(W)
        generate_names(W_hat, 5)

At step: 0, loss: 7.715683460235596
म्पीवेर्कक्शूक्षीसैष्काक्सेभौम्हम्भुपँसौमूंव्राल्टहाँप्तैद्धोव्यक्षन्हॅूल्मीन्टाब्बूर्टच्छुध्दिल्बेछॅदिछाँऋृम्वान्साब्दुस्सिन्हूर्रूहौच्चोल्कामूँधूकौर्बाङीझूर्योन्कूभंडेद्वेच्छाणात्येड्डूशून्डेठद्दिठिन्डय्यर्तमन्टाव्तेन्हूद्वीम्पाडॅूतुंत्थूस्मिश्यन्द्राल्हेअंद्दीज्जेचश्लाइंगूज्जुनून्धीस्थिलेगृध्धूप्तैध्दुबाँटंबूीव्यानंवंय्याडूढाम्मीय्यूद्धराआेग्याढ़ाक्सात्तूक्रिक्लान्जूर्थीन्थूमौब्लिन्छग्धूकृसांर्वश्व्नाशैख्मीभन्हिन्घपाँलिंल्तार्फुसुंचिंतोहीठुझोष्टित्कश्रींम्ताबैन्तिूच्चिबोछौप्याश्माहव्दून्चूस्त्याषूर्भयाद्याढ़ाफ्तियूम्बीम्भिबुड़ीष्टर्दोर्लीशँध्धूडि़फुल्हझंतैवंश्याडँणिन्दिर्चिल्पीस्थिस्मिभौर्रूशांघ्घूव्यवाँम्हुम्बीडींघेख्तसीजँत्मघिप्रांल्तात्तेधौंक्खीनंन्छेत्तेम्त्याफ्तन्हूंब्लिम्मून्हाँन्धून्झान्ताक्याबृम्वच्छेम्देस्थाऋतिर्देस्सोफ्फूश्योकोहुंन्डोढैद्वेद्धात्सफ्ताकृपितंस्फाहींवँल्हेन्तीुरेस्मल्हाभैरील्टीट्रल्पीक्सीश्फाक्टल्फेर्इंच्छाक्ष्मन्द्रूव्यल्लीन्झून्वाय्यास्फामोल्फाड्डुय्यार्जीझिंड्डेभँर्जात्नेताँन्सून्दव्यक्मटैम्बेन्दूर्णेदंचंविंगाणन्हांड्डल्

ण्टछोटैन्चूम्तेशोंस्पम्मेझाँदोंत्थकांण्डैडूँक्फर्दाब्दश्म्भक्रिर्णप्पन्यम्पाफाध्यडूॅभूपुतेड्डासुंद्धस्फान्थज्जुटोदग्गुलिंमूॅम्तखीन्हत्फन्शुन्तिूटैम्बोन्गूखुव्वीन्थह्मल्जोबूंगीक्षैर्गुसौध्याद्वेफौख्यान्सुम्भूच्छून्टेर्हश्रास्सोल्झचिऋृन्हाँन्हिदाएगन्छाचॉस्टद्वेश्याट्टोगम्त्याबँर्येफुप्तेन्त्याक्मक्सछिंगीफैलौल्लून्जूीहूर्नम्मिरांघुन्दूस्थाड़ैल्तिढ़बूंओंन्हॉदिेवीजट्टोन्धील्काट्टुदिेक्सीटेशीबेढिधीर्यछिधाँग्रिघीयोर्मटिस्हाखैनृन्ध्वाखेसाँदाध्दिक्राष्मिन्द्रिइंक्मार्नष्नूभंगोग्बिर्णढि़त्यूद्धुऊंफ्ताश्रींतूीटंतग्गीम्बूख्यौन्तीूठूल्झश्व्नान्हौद्रिनीन्घ्याथीभौफुत्पाखाम्मासातांक्शफ्फख्तल्फिखूकांल्ताव्याद्धेदन्छेप्रित्तिन्ध्यांअंल्डीक्खम्वीछेजूरौंण्टष्नूर्बआेष्पग्धूलौंष्यदाबान्नोढैम्मारिेहेक्सूभुह्रण्णफूथुशैर्पिश्लाण्डपोर्जूषीफुतिूग्रेस्यमिफोच्चीष्पन्यअँड्डूक्खून्जीत्सल्वेन्डूयूखीम्ब्रीन्कीब्बल्ठूज्जैम्बान्शात्थूट्रभोजूल्हाकंख्वान्तुअंटंध्रुढ़ोक्मर्सशधौवौझांफ्फूक्शीमिंमेशामींढैलैम्बीन्दृक्ष्मीप्रिस्कजेकोर्शीन्सार्यद्धीण्डूम्नक्षेट्ठन्जूीक्ष्यगंर्णिर्योर्गेफिन्धूध्रुखांन्दिख्ताल्लाअँजपिद्वोठूछंहैन्ह

ल्मिन्दाढोक्तल्लीबुन्डीक्छेटिन्द्रीम्बून्हिच्छोडूॅख्यल्होथ्वीरौंख्खुत्रुणुट्ठूचेर्तीक्तील्कश्तिन्झून्चीप्फाशेषूसुखेह्मून्ठूक्फग्रिधंगुंस्नाड्डाध्धूद्वाच्छफौत्रम्ताडीद्रीन्कस्जिहृहिमाँत्तूगृल्पीमूँवस्माग्गुन्हौखोध्दिद्धाणाज्जेम्विफ्फाल्यछाडिश्तेजूीस्वेल्फिग्यवंल्पाश्वाद्धकँबदोंच्छासांन्जेहॅच्छेम्मायुश्रृंर्रूहृच्चात्थोन्धुप्फस्लिट्ठुरौंभुयाँआैप्तास्थिचांण्डोझौभषून्युख्खूश्म्भर्बिट्ठूब्लीन्स्पेठाहौंसोज्जोझौझाँख्खुस्हार्सीप्रेओरैग्गुम्रल्कम्खांन्काध्नस्सुन्छूशोंओंस्तार्तान्झास्मानिंर्सगंषीसृपंशीठाप्पठोद्वीमुपँन्ध्याण्डोक्षुर्वेम्वीरेढोंछोव्यातिूश्लोन्सीन्हाँम्म्दनून्ठद्वक्छेश्लाड़ैभीझेद्दीदोंटंवंद्धूच्चीसेर्फीवीढ़ार्फुह्लाब्रेक्रुक्कूक्ष्मीर्शीझांन्सानमूँबांस्दीकुअँत्मष्यंक्रेर्राम्ताप्रांब्बाप्तिम्त्याम्खांरुम्हथ्रन्जूीर्बअौल्पीम्बुकृडॅूद्रिप्रीजोभेठुगंब्रेछेक्रेझॉघून्धीक्षूदंभ्मश्रेप्तिम्पिश्मक्षेन्तीुल्तन्साश्रुदैख्यल्हौढुघीच्छाशुऐष्णान्छोन्हूवॉस्वाध्रुड्डीर्दोन्कन्शच्युश्बन्द्रिख्यौन्जूीडूँक्सष्मिश्तात्नाक्रसांतिूष्टिर्दासैन्दृथ्बीश्नूद्वीच्छक्शूम्पीन्जम्पाख्याठीकोह्रभ्मतिधंमाश्नाश्बक़

At step: 300, loss: 4.360914707183838
रामुर्फीक्फक्षासषेक्ष्मीम्पम्मुमीद्वूसॉम्ब्रीव्दूल्डीहादुमुश्यक्षान्यालेथ।
लाक्राध्यील्हघूत्थुज्जैआंम्दपीबांष्ण्पान्सार्मोहुँछैलिंमुँध्रुक्रर्मच्छेटार्मेन्घ्यामाँडुद्वुचिण्डूदुन्द्रील्होन्कछूद्धेडेन्चारिेर्फीम्बिन्टोव्वीन्हैद्दिरिथोत्रण्टूस्त्रीशोंन्तीब्बेअॅत्रीज।
सा।
कखोन्घ्यापंस्यलौंगूद्धूप्रिंध्दबांटांब्बूल्देद्वोल्पूम्भाढड़म्बूम्विधंयेउडूंडिक्शटिंक्यूख्यान्डर्इंन्तूबँबूीनौंघुकोष्णाल्काश्राशा।
मुन्नीद्वूप्पेर्षालोर्धष्पाङीण्णहैट्टुझीष्णूतोश्तिम्भिबाँन्त।
At step: 400, loss: 4.2214674949646
रामार।
इसष्काक्सेभौम्हम्भुपँसौमूँवौल्ठूहांन्हादोल्लक्षेन्हौल्कन्चीफीर्नाच्युधांल्हीचाँतूीछोई।
भुफ्तसिहमहेच्चील्मिम्देद्धाक्मर्बाङीझूर्योन्कूभंडैन्कीज्ञढीथीडूवँन्ती।
ना।
प्रसान्कीयाकली।
At step: 500, loss: 4.119882106781006
रंल्हारु।
इसश्हाक्शीमिंम्भूम्मान्दूस्तिमैव्दाल्टूहुब्रादया।
बोवंन्स्पेब्बार्मचूध्वल्मिचौढिझऔयोन्हांफ्फासंन्नुलह।
विभून्जूकून्तिूचूझीर्गीनिर।
नूीच्चुठूत्नेढेवॉन्जाटड़िडेन्घ्यारैर्दभोन्ड़ेशंकन्जूीम्बधौतौत्नस्नाश्योंन्तब्लिअफअजीव।
At step: 600,

KeyboardInterrupt: 